In [ ]:
import openai,files,fitz

In [ ]:
openai.api_key = files.openaikey

# Masking

authenticating...

In [ ]:
doc = fitz.open(files.pdf_path)
doc.authenticate(files.password)
new=fitz.open()
page_number = 0  
text = doc[page_number].get_text()

Extracting the confidential data from the text

In [ ]:
def ner(input_text):
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=f"Extract the entities from the text with labels name,emailid,phone number,password,account number\ntext: Ram's email is Ram2267@example.com and his phone number is 1234567890. His Account number is A11d345. Ram has a younger sister names Saaya\n\nId:1\nRam:name\nRam2267@example.com:Email\n1234567890:phone number\nnull:Password\nA11d345:Account number\n##\ntext: Anu's email is anu88@example.com and her phone number is 9999834521. Hes Account number is B54ERop\n\nId:2\nAnu:Name\nanu88@example.com:Email\n9999834521:phone number\nnull:Password\nB54ERop:Account number\n##\ntext:{input_text}\n",
        temperature=0.02,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    extracted_text = response['choices'][0]['text']

    # Extract the entities from the extracted text and create a dictionary
    entities = {}
    # print(extracted_text)
    for line in extracted_text.split('\n'):
        if ':' in line:
            key, value = line.split(':')
            entities[key.strip()] = value.strip()

    return entities

In [ ]:
original_entities = ner(text)
original_entities

Creating a dictionary to store original data and synthetic data correspondence


In [ ]:
correspondence_dict = {}

generating synthetic data

In [ ]:
for entity, entity_type in original_entities.items():
    prompt1 = f"Generate a synthetic replacement for the {entity_type.lower()}: '{entity}'."
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt1,
        max_tokens=50
    )
    synthetic_data = response.choices[0].text.strip()
    correspondence_dict[entity] = synthetic_data

In [ ]:
correspondence_dict

Masking the original text information

In [ ]:
masked_text = text
for org_data, syn_data in correspondence_dict.items():
    masked_text = masked_text.replace(org_data, syn_data)

In [ ]:
masked_text

saving to pdf

In [ ]:
modified_image = doc[page_number].get_pixmap(matrix=fitz.Matrix(1, 1), clip=doc[page_number].rect, alpha=False)
new_page = new.new_page(width=doc[page_number].rect.width, height=doc[page_number].rect.height)
new_page.insert_text((10, 10),masked_text, fontname="helv", fontsize=12)
new.save('masked.pdf')

In [ ]:
doc.close()
new.close()

# Unmasking

In [ ]:
doc = fitz.open('masked.pdf')
# doc.authenticate(files.password)
new=fitz.open()
page_number = 0  
text = doc[page_number].get_text()

In [ ]:
text

In [ ]:
unmasked_text = text
for org_data, syn_data in correspondence_dict.items():
   unmasked_text = unmasked_text.replace(syn_data,org_data)

In [ ]:
unmasked_text

In [ ]:
modified_image = doc[page_number].get_pixmap(matrix=fitz.Matrix(1, 1), clip=doc[page_number].rect, alpha=False)
new_page = new.new_page(width=doc[page_number].rect.width, height=doc[page_number].rect.height)
new_page.insert_text((10, 10),unmasked_text, fontname="helv", fontsize=12)
new.save('unmasked.pdf')